# Installing dependencies

In [ ]:
!pip3 install openmeteo-requests
!pip3 install requests-cache retry-requests numpy pandas
!pip3 install paho-mqtt

# Open Meteo Script. Only for code understanding

In [2]:
#Upload de open meteo script to the local system
#Open-meteo fetch the meteo data

import openmeteo_requests

import requests_cache
import pandas as pd
from retry_requests import retry

def get_data_meteo(lat, lon, start_date, end_date):
    """Obtiene datos meteorológicos históricos desde Open-Meteo y devuelve un DataFrame."""
    cache_session = requests_cache.CachedSession('.cache', expire_after=-1)
    retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
    openmeteo = openmeteo_requests.Client(session=retry_session)

    url = "https://archive-api.open-meteo.com/v1/archive"
    params = {
        "latitude": lat,
        "longitude": lon,
        "start_date": start_date,
        "end_date": end_date,
        "hourly": ["temperature_2m", "relative_humidity_2m", "dew_point_2m"],
        "timezone": "Europe/Berlin"
    }

    responses = openmeteo.weather_api(url, params=params)
    response = responses[0]

    # Procesar datos horarios
    hourly = response.Hourly()
    hourly_data = {
        "date": pd.date_range(
            start=pd.to_datetime(hourly.Time(), unit="s", utc=True),
            end=pd.to_datetime(hourly.TimeEnd(), unit="s", utc=True),
            freq=pd.Timedelta(seconds=hourly.Interval()),
            inclusive="left"
        ),
        "temperature_2m": hourly.Variables(0).ValuesAsNumpy(),
        "relative_humidity_2m": hourly.Variables(1).ValuesAsNumpy(),
        "dew_point_2m": hourly.Variables(2).ValuesAsNumpy()
    }

    return pd.DataFrame(hourly_data)

ModuleNotFoundError: No module named 'openmeteo_requests'

# Sender.py Programm modified for running hardwareless

In [5]:
#Sender.py modified

import time
import module_openmeteo

import paho.mqtt.client as paho
from paho import mqtt

def on_publish(client, userdata, mid):
    print("mid: "+str(mid))

def on_subscribe(client, userdata, mid, granted_qos, properties=None):
    print("Subscribed: " + str(mid) + " " + str(granted_qos))

def on_message(client, userdata, msg):
    global espera

    print(msg.topic + " " + str(msg.qos) + " " + str(msg.payload))
    espera = msg.payload

def sensor_inside(df, reg):
    global temperature_inside, humidity_inside, dew_point_inside

    temperature_inside = df.iloc[reg]["temperature_2m"]
    humidity_inside  = df.iloc[reg]["relative_humidity_2m"]
    dew_point_inside  = df.iloc[reg]["dew_point_2m"]

    print(temperature_inside, humidity_inside, dew_point_inside)
    print("data_inside complete")

temperature_inside = 0
humidity_inside = 0
dew_point_inside = 0
espera = 1
reg = 0

# Calculo el número de registros descargados
df = module_openmeteo.get_data_meteo(26.2907, 50.178, "2025-01-01", "2025-02-13")
num_reg = df.shape[0]

client = paho.Client()
client.on_publish = on_publish
client.tls_set(tls_version=mqtt.client.ssl.PROTOCOL_TLS)
client.username_pw_set("galeoiot-dev", "3J2Dj3p7kHDz")
client.connect("1a72e178acf1458a8a6814ac97606398.s2.eu.hivemq.cloud", 8883)
client.loop_start()

client.on_message = on_message
client.on_subscribe = on_subscribe
client.subscribe("cmdn/sampling", qos=0)

while True:
    try:
        reg = reg + 1
        if reg > num_reg:
            reg = 0
        sensor_inside(df, reg)

        (rc, mid) = client.publish("telemetry/temperature_indoor", str(temperature_inside), qos=0)
        (rc, mid) = client.publish("telemetry/humidity_indoor", str(humidity_inside), qos=0)
        (rc, mid) = client.publish("telemetry/dew_point_indoor", str(dew_point_inside), qos=0)

    except RuntimeError as error:
        # Errors happen fairly often, DHT's are hard to read, just keep going
        print(error.args[0])
        time.sleep(2.0)
        continue
    except Exception as error:
        raise error

    time.sleep(int(espera))

<ipython-input-5-0bb7a81fa74e>:41: DeprecationWarning: Callback API version 1 is deprecated, update to latest version
  client = paho.Client()


16.417501 57.94952 8.117499
data_inside complete
mid: 2
mid: 3
mid: 4
Subscribed: 1 (0,)
16.2675 53.721558 6.8675003
data_inside complete
mid: 5
mid: 6
mid: 7
15.6675 49.63658 5.1675
data_inside complete
mid: 8
mid: 9
mid: 10
15.117499 50.180866 4.8175
data_inside complete
mid: 11
mid: 12
mid: 13
14.6675 50.408936 4.4675
data_inside complete
mid: 14
mid: 15
mid: 16
15.367499 49.03847 4.7175
data_inside complete
mid: 17
mid: 18
mid: 19
17.067501 43.697086 4.6175003
data_inside complete
mid: 20
mid: 21
mid: 22
18.917501 38.09184 4.3175
data_inside complete
mid: 23
mid: 24
mid: 25
19.667501 32.825447 2.8675
data_inside complete
mid: 26
mid: 27
mid: 28
20.1175 30.37421 2.1675
data_inside complete
mid: 29
mid: 30
mid: 31
20.3675 27.644447 1.0675
data_inside complete
mid: 32
mid: 33
mid: 34
20.2175 26.91474 0.5675
data_inside complete
mid: 35
mid: 36
mid: 37
19.667501 23.550331 -1.7325001
data_inside complete
mid: 38
mid: 39
mid: 40
19.167501 25.480686 -1.0825
data_inside complete
mid: 41
mi

KeyboardInterrupt: 